In [1]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 83c7a2dd-bef5-47dc-bc6b-3a4b432d84e8

In [2]:
import vision as vis
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos
import math
import motion_control as motion



COMPUTE = 0
TRAVEL = 1
LOCAL_NAV = 2
FINISH = 3

In [ ]:
# Main -> state machine
state = COMPUTE
obstacle = False
path_calculated = False

cap = cv.VideoCapture(2)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while True:
    time.sleep(0.01)
    if state == COMPUTE:
        captured, img = cap.read()
        if captured:
            cropped, cropped_img = vis.crop_map(img)
            # print(cropped_img.shape)        # uncomment to debug
            # cv.imshow("Original", img)        # uncomment to debug
            cv.imshow("Cropped", cropped_img)
            if cropped:
                im3 = np.zeros(cropped_img.shape)
                found, obstacles = vis.extract_obstacles(cropped_img)
                if found:
                    ex_obstacles = vis.expand_obstacles(obstacles)
                    cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                    cv.imshow("expanded", im3)
                found_rob, robot_pos = pos.get_robot_position(cropped_img)
                if found_rob:
                    vis.annotate_robot(robot_pos, im3)
                found_goal, goal_pos = pos.get_goal_position(cropped_img)
                if found_goal:
                    vis.annotate_goal(goal_pos, im3)
                found_arch, arch_pos = pos.get_arch_positions(cropped_img)
                if found_arch and found_rob:
                    if math.dist(arch_pos[0],robot_pos[0])>math.dist(arch_pos[1],robot_pos[0]):
                        arch_pos = np.flip(arch_pos,0)
                    vis.annotate_arch(arch_pos, im3)
                if (found and found_rob and found_goal and found_arch):
                    if path_calculated == False:
                        vertices = vis.convert_vertice(ex_obstacles)
                        pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(arch_pos[0]))
                        path_arch =\
                        short.names_to_subpaths(pathname,vertices,tuple(robot_pos[0]), tuple(arch_pos[0]))[0]
                        vis.annotate_path(path_arch,im3)

                        pathname2 = short.find_shortest_path(vertices, tuple(arch_pos[1]), tuple(goal_pos))
                        path_goal =\
                        short.names_to_subpaths(pathname2,vertices,tuple(arch_pos[1]), tuple(goal_pos))[0]
                        vis.annotate_path(path_goal,im3)
                        cv.imshow("environment", im3)
                        state = TRAVEL
        else:
            print("There was a problem in the capture")
            break
    if state == TRAVEL:
        cv.imshow("environment", im3)
        cv.waitKey(0)
        for subgoal in path_arch[1:]:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                last_pos = robot_pos
                success, robot_pos = pos.get_robot_position(cropped_travel)
                if not success:
                    robot_pos = last_pos
                    print("thymio disappeared")
                im4 = np.zeros(cropped_travel.shape)
                vis.annotate_robot(robot_pos,im4)
                vis.annotate_path(path_arch,im4)
                print("robot angle", robot_pos[1])
                err = motion.get_error(robot_pos,subgoal)
                cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.imshow("mission status", im4)
                cv.waitKey(10)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                print("error",err)
                print("motor_left",motor_left)
                print("motor_right",motor_right)

                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        for subgoal in path_goal:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                last_pos = robot_pos
                success, robot_pos = pos.get_robot_position(cropped_travel)
                if not success:
                    robot_pos = last_pos
                err = motion.get_error(robot_pos,subgoal)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                print("error",err)
                print("motor_left",motor_left)
                print("motor_right",motor_right)

                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        state = FINISH
    if state == FINISH:
        v = {
                    "motor.left.target": [0],
                    "motor.right.target": [0],
                }
        aw(node.set_variables(v))
            #print("motor_left",motor_left)
            #print("motor_right", motor_right)
            #motor_left_target = int(motor_left)
            #motor_right_target = int(motor_right)
            #print("motor_left_t",motor_left_target)
            #print("motor_right_t", motor_right_target)          
    cv.imshow("environment",im3)
    #cv.waitKey(0)
cap.release()
print("finished!")
# cv.destroyAllWindows()
    

"""
# robot run    
elif short.find_shortest_path() == True:
    state = 1

obstacle = check_obstacle(prox_horizontal)
    
        # global navigation
        if  obstacle == False:
            motion_control

        # local navigation
        elif obstacle == True:
"""
            
            
        
    

C:\Users\sebas\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:804: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


Start (156, 66)
Goal (352, 146)
Vertices [[(190, 292), (357, 324), (317, 443), (271, 445)], [(63, 163), (198, 250), (136, 339), (97, 348), (-6, 255)], [(436, 53), (497, 71), (435, 278), (368, 277)], [(183, 0), (244, 111), (166, 151), (215, 120), (163, 8)]]
list neighbours [[(5, 134.3800580443393), (16, 85.58621384311844), (17, 79.98124780221924), (18, 58.42088667591412)], [(2, 170.03823099526764), (4, 173.11845655504212), (6, 42.7551166528639), (7, 71.58910531638176), (10, 342.9825068425502), (12, 245.39967400141344), (13, 178.63090438107287), (15, 188.88356201639147), (19, 218.08255317654368)], [(1, 170.03823099526764), (3, 125.54282137979854), (5, 335.19695702676063), (6, 175.37673733993344), (12, 90.55385138137417), (13, 48.27007354458868), (15, 241.11822826157297), (16, 257.70137756713683), (19, 178.07021087200408)], [(2, 125.54282137979854), (4, 46.04345773288535), (12, 202.85216291674092)], [(1, 173.11845655504212), (3, 46.04345773288535), (7, 171.64206943520577), (8, 199.2109434

robot angle -1.0584068664841588
first if
error -0.3721398506741327
motor_left 150
motor_right 150
robot angle -1.0808390005411683
first if
error -0.37005334088870434
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.0808390005411683
first if
error -0.37005334088870434
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.0808390005411683
first if
error -0.37005334088870434
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.0808390005411683
first if
error -0.37005334088870434
motor_left 150
motor_right 150
robot angle -1.0303768265243125
first if
error -0.48251329502247686
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.0303768265243125
first if
error -0.48251329502247686
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.0303768265243125
first if
error -0.48251329502247686
motor_left 150
motor_right 150
robot angle -1.0584068664841588
first if
error -0.5462747902148555
motor_left 150
motor_right 150
thymio disappear

first if
error -0.31667874665915763
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.9505468408120752
first if
error -0.31667874665915763
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.9505468408120752
first if
error -0.31667874665915763
motor_left 150
motor_right 150
robot angle -0.8621700546672264
first if
error -0.5204047668228995
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.8621700546672264
first if
error -0.5204047668228995
motor_left 150
motor_right 150
robot angle -0.9048270894157867
first if
error -0.5034801281619065
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.9048270894157867
first if
error -0.5034801281619065
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.9048270894157867
first if
error -0.5034801281619065
motor_left 150
motor_right 150
robot angle -0.9048270894157867
first if
error -0.5034801281619065
motor_left 150
motor_right 150
robot angle -0.982793723247329
first if
error -0.461

thymio disappeared
robot angle -1.9100889412489412
second if
error 0.4464079378062764
motor_left -183
motor_right 183
thymio disappeared
robot angle -1.9100889412489412
second if
error 0.4464079378062764
motor_left -183
motor_right 183
robot angle -2.0106389096106327
second if
error 0.46134464480196735
motor_left -185
motor_right 185
thymio disappeared
robot angle -2.0106389096106327
second if
error 0.46134464480196735
motor_left -185
motor_right 185
robot angle -1.6295521495106193
error 0.08073030701238526
motor_left 150
motor_right 150
robot angle -1.4056476493802699
first if
error -0.14364661542839552
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.4056476493802699
first if
error -0.14364661542839552
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.4056476493802699
first if
error -0.14364661542839552
motor_left 150
motor_right 150
robot angle -1.3521273809209546
first if
error -0.1716448140311957
motor_left 150
motor_right 150
thymio disappeared
robo

...

thymio disappeared
robot angle -0.6659692373791098
first if
error -0.9925137445452873
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.6659692373791098
first if
error -0.9925137445452873
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.6659692373791098
first if
error -0.9925137445452873
motor_left 150
motor_right 150
robot angle -2.5213431676069717
second if
error 0.970746113393143
motor_left -223
motor_right 223
robot angle -2.5535900500422257
second if
error 0.9621780876217254
motor_left -222
motor_right 222
robot angle -2.396173177315635
second if
error 0.8253768505207386
motor_left -212
motor_right 212
thymio disappeared
robot angle -2.396173177315635
second if
error 0.8253768505207386
motor_left -212
motor_right 212
robot angle -1.9295669970654687
second if
error 0.337268608284341
motor_left -175
motor_right 175
thymio disappeared
robot angle -1.9295669970654687
second if
error 0.337268608284341
motor_left -175
motor_right 175
thymio disappeared
rob

thymio disappeared
robot angle -1.2120256565243244
first if
error -0.5324418686121462
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.2120256565243244
first if
error -0.5324418686121462
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.2120256565243244
first if
error -0.5324418686121462
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.2120256565243244
first if
error -0.5324418686121462
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.2120256565243244
first if
error -0.5324418686121462
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.2120256565243244
first if
error -0.5324418686121462
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.2120256565243244
first if
error -0.5324418686121462
motor_left 150
motor_right 150
thymio disappeared
robot angle -1.2120256565243244
first if
error -0.5324418686121462
motor_left 150
motor_right 150
robot angle -2.2367655641740063
second if
error 0.8657978824521


robot angle 1.5152978215491797
second if
error 1.576336436318671
motor_left -268
motor_right 268
thymio disappeared
robot angle 1.5152978215491797
second if
error 1.576336436318671
motor_left -268
motor_right 268
thymio disappeared
robot angle 1.5152978215491797
second if
error 1.576336436318671
motor_left -268
motor_right 268
robot angle 2.356194490192345
second if
error 0.7681584922029185
motor_left -208
motor_right 208
thymio disappeared
robot angle 2.356194490192345
second if
error 0.7681584922029185
motor_left -208
motor_right 208
thymio disappeared
robot angle 2.356194490192345
second if
error 0.7681584922029185
motor_left -208
motor_right 208
robot angle 3.0309354324158977
error 0.021607638539397733
motor_left 150
motor_right 150
robot angle -3.0828368308740703
second if
error 6.135379901829365
motor_left -610
motor_right 610
robot angle -2.9669204545815537
second if
error 6.016183773439748
motor_left -601
motor_right 601
robot angle -2.677945044588987
second if
error 5.68572672

thymio disappeared
robot angle 1.396124127786657
first if
error -1.960693521454757
motor_left 150
motor_right 150
thymio disappeared
robot angle 1.396124127786657
first if
error -1.960693521454757
motor_left 150
motor_right 150
thymio disappeared
robot angle 1.396124127786657
first if
error -1.960693521454757
motor_left 150
motor_right 150
thymio disappeared
robot angle 1.396124127786657
first if
error -1.960693521454757
motor_left 150
motor_right 150
robot angle 1.4056476493802699
first if
error -2.399251762301706
motor_left 150
motor_right 150
thymio disappeared
robot angle 1.4056476493802699
first if
error -2.399251762301706
motor_left 150
motor_right 150
robot angle 1.460139105621001
first if
error -2.5594754823549906
motor_left 150
motor_right 150
thymio disappeared
robot angle 1.460139105621001
first if
error -2.5594754823549906
motor_left 150
motor_right 150
thymio disappeared
robot angle 1.460139105621001
first if
error -2.5594754823549906
motor_left 150
motor_right 150
robot a

first if
error -0.2440803925398113
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.3392926144540447
first if
error -0.2440803925398113
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.3392926144540447
first if
error -0.2440803925398113
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.3392926144540447
first if
error -0.2440803925398113
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.3392926144540447
first if
error -0.2440803925398113
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.3392926144540447
first if
error -0.2440803925398113
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.3392926144540447
first if
error -0.2440803925398113
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.3392926144540447
first if
error -0.2440803925398113
motor_left 150
motor_right 150
thymio disappeared
robot angle -0.3392926144540447
first if
error -0.2440803925398113
motor_left 150
motor_right 15

In [ ]:
OBSTACLE_THRESHOLD 10

def check_obstacle(prox_horizontal):
    prox_values = [prox_horizontal[0], prox_horizontal[4]]
    if prox_values[0] > OBSTACLE_THRESHOLD or prox_values[1] > OBSTACLE_THRESHOLD:
        obstacle = True
        
    else:
        obstacle = False
        
        
    